<a href="https://colab.research.google.com/github/filipp134/Telegram-bot-TMDB-movies/blob/main/TMDB_Bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install python-telegram-bot python-telegram-bot
from datetime import date, timedelta, datetime
import requests
from telegram import InlineKeyboardButton, InlineKeyboardMarkup, Update, ParseMode
from telegram.ext import Updater, CommandHandler, MessageHandler, Filters, CallbackQueryHandler, CallbackContext, \
    ConversationHandler

TOKEN = '1548279453:AAH3pAQNNoKA8GeYr8Zx674DlXxAitPmm3g'
API_KEY = 'e3bd30e9ec6db1013021ef223e7b39ae'
API_LINK = 'https://api.themoviedb.org/3'
IMAGES_LINK = 'https://image.tmdb.org/t/p/original'
MOVIE_LINK = 'https://www.themoviedb.org/movie/'
SEARCH_LIST, TRENDING_LIST = range(2)
PREV, NEXT = range(2)
HEADERS = {'Content-Type': 'application/json;charset=utf-8'}


def keyboard_forward_backward():
    """Генерируем инлайн клавиатуру"""
    keyboard = [
        [
            InlineKeyboardButton("<", callback_data=str(PREV)),
            InlineKeyboardButton(">", callback_data=str(NEXT)),
        ]
    ]
    return InlineKeyboardMarkup(keyboard)



def start(update, context):
    """Начинаем общение с ботом"""
    context.bot.send_message(chat_id=update.effective_chat.id, text="I'm a bot, please talk to me!")
    return ConversationHandler.END


def trending(update: Update, context: CallbackContext) -> None:
    """ Получение топа фильмов по популярности из TMDB в России для телеграм-бота.  """
    payload = {
        'api_key': API_KEY,
        'region': 'RU',
        'sort_by': 'popularity.desc',
        # выход в прокат от двух недель назад и до сегодня
        'primary_release_date.gte': (date.today() - timedelta(days=14)).isoformat(),
        'primary_release_date.lte': date.today().isoformat(),
        'language': 'ru',
    }
    trends = requests.get(f'{API_LINK}/discover/movie', params=payload).json()

    if not trends['results']:
        context.bot.send_message(chat_id=update.effective_chat.id, text="Похоже, что фильмы не в тренде")
        return ConversationHandler.END  # Заканчиваем ветку разговора

    # Добавляем id фильмов во внутреннее хранилище бота
    context.chat_data['films_ids'] = list()
    trend_info = str()
    i = 1
    # Ограничиваем 10ю результатами
    for film in trends['results'][:10]:
        # Здесь отформатировал вывод программы
        trend_info += (
            # Добавил ссылки для выбора фильма и описание
            f'/{i} {film["title"]}\n'
            f'{film["release_date"]} | {film["original_language"]} | {str(film["vote_average"])}\n'
            f'{film["overview"][:100]}...\n\n'
        )
        context.chat_data['films_ids'].append(film['id'])
        i += 1
    context.bot.send_message(chat_id=update.effective_chat.id, text=trend_info)
    return TRENDING_LIST


def find_film(update, context):
    movie_name = update.message.text
    payload = {
        'api_key': API_KEY,
        'query': movie_name,
    }
    film_info = requests.get(f'{API_LINK}/search/movie', params=payload).json()

    # Добавил проверку на случай если результаты не будут найдены
    if not film_info['results']:
        context.bot.send_message(chat_id=update.effective_chat.id, text="Фильмов с таким названием не найдено!")
        return ConversationHandler.END

    # Сохраняем в хранилище страницы, текущую страницу и название фильма, который ищем
    context.chat_data['pages'] = range(1, film_info["total_pages"] + 1)
    context.chat_data['find_page'] = 0
    context.chat_data['movie_name'] = movie_name

    movie_info = str()
    context.chat_data['films_ids'] = list()
    for film in film_info['results']:
        movie_info += (
            f"/{film_info['results'].index(film) + 1} {film['title']}\n"
            f"{film['release_date']} | {film['original_language']} | {str(film['vote_average'])}\n"
            f"{film['overview'][:100]}...\n\n"
        )
        context.chat_data['films_ids'].append(film['id'])
    movie_info += f'{film_info["page"]}/{film_info["total_pages"]}'
    context.bot.send_message(chat_id=update.effective_chat.id, text=movie_info,
                             reply_markup=keyboard_forward_backward())
    return SEARCH_LIST


def find_film_callback(update, context):
    """Callback для обновлении сообщения при переключении страницы"""
    movie_name = context.chat_data['movie_name']
    query = update.callback_query
    # Необходимо ответить на запрос пользователя
    query.answer()
    pages = context.chat_data['pages']
    if query.data == str(NEXT):
        context.chat_data['find_page'] += 1
    elif query.data == str(PREV):
        context.chat_data['find_page'] -= 1
    # Избегаем ошибки out of range
    if context.chat_data['find_page'] >= len(pages) \
            or context.chat_data['find_page'] <= -len(pages):
        context.chat_data['find_page'] = 0
    payload = {
        'api_key': API_KEY,
        'query': movie_name,
        'page': pages[context.chat_data['find_page']],
    }
    film_info = requests.get(f'{API_LINK}/search/movie', params=payload).json()
    if not film_info['results']:
        context.bot.send_message(chat_id=update.effective_chat.id, text="Фильмов с таким названием не найдено!")
        return ConversationHandler.END
    context.chat_data['films_ids'] = list()
    movie_info = str()
    for film in film_info['results']:
        movie_info += (
            f"/{film_info['results'].index(film) + 1} {film['title']}\n"
            f"{film['release_date']} | {film['original_language']} | {str(film['vote_average'])}\n"
            f"{film['overview'][:150]}...\n\n"
        )
        context.chat_data['films_ids'].append(film['id'])
    movie_info += f'{film_info["page"]}/{film_info["total_pages"]}'
    query.edit_message_text(
        text=movie_info, reply_markup=keyboard_forward_backward()
    )
    return SEARCH_LIST


def film_details(update, context):
    element_number = update.message.text
    film_id = context.chat_data['films_ids'][int(element_number[1:]) - 1]
    payload = {
        'api_key': API_KEY,
        'language': 'ru',
    }
    film = requests.get(f'{API_LINK}/movie/{film_id}', params=payload).json()
    reply_markup = None
    film_info = (
        f'{film["title"]}\n'
        f'{film["release_date"]} | {film["original_language"]} | {str(film["vote_average"])}\n'
        f'{film["overview"]}\n\n'
        f'<a href="{MOVIE_LINK + str(film_id)}">Ссылка на TMDB</a>'
    )
    try:
        # Если возможно - отправляем постер с текстом
        context.bot.send_photo(chat_id=update.effective_chat.id, photo=IMAGES_LINK + film['poster_path'],
                               caption=film_info,
                               parse_mode=ParseMode.HTML)
    except TypeError:
        # Если нет -
        context.bot.send_message(chat_id=update.effective_chat.id, text=film_info,
                                 parse_mode=ParseMode.HTML, reply_markup=reply_markup)
    return ConversationHandler.END


def main():
    updater = Updater(token=TOKEN, use_context=True)
    dispatcher = updater.dispatcher

    # Добавляем ConversationHandler для умного управления общением с ботом
    conv_handler = ConversationHandler(
        entry_points=[
            MessageHandler(Filters.text & (~Filters.command), find_film),
            CommandHandler('trending', trending),
        ],
        states={
            SEARCH_LIST: [
                CallbackQueryHandler(find_film_callback),
                CommandHandler([str(c) for c in range(1, 21)], film_details),
            ],
            TRENDING_LIST: [
                CommandHandler([str(c) for c in range(1, 21)], film_details),
            ]
        },
        fallbacks=[
            CommandHandler('start', start)
        ])

    dispatcher.add_handler(conv_handler)
    updater.start_polling()


main()

     |████████████████████████████████| 450kB 5.9MB/s 
     |████████████████████████████████| 61kB 6.3MB/s 


/usr/local/lib/python3.7/dist-packages/telegram/ext/conversationhandler.py:280: UserWarning: If 'per_message=False', 'CallbackQueryHandler' will not be tracked for every message.
  "If 'per_message=False', 'CallbackQueryHandler' will not be "
